In [1]:
library(tidyverse)
library(repr)
library(digest)


Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.8     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.1
✔ readr   2.1.2     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [30]:
#Question: do ckd patients have the same level of albumin as non-ckd patients?

#Patients without ckd could have other diseases - maybe diseases which actually affects the liver
#See if ckd has the same affect on albumin as other potential diseases
#does not say if ckd patients stictly have ckd
#An observation which pertains to this sample (maybe)


#reading the data
dataset <- read.csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00624/ckd-dataset-v2.csv", skip = 2) 

#wrangling data
ckd_data <- dataset %>%
    rename(albumin_level = X.3, classification = X.4, appetite = X.22)%>% #renaming columns
    mutate(albumin_level= str_replace_all(albumin_level, c("< 0" = "0", "1 - 1" = "1", "2 - 2" = "2", "3 - 3" = "3", "≥ 4" = "4")))%>% #turning strs into numbers    mutate(albumin_level = as.numeric(albumin_level))%>%
    select(albumin_level, classification, appetite)%>% #selecting relevant variables
    mutate(albumin_level = as.numeric(albumin_level))%>%#change chr into dbl for albumin_level
    mutate(appetite = as.factor(appetite)) #appetite may be another interesting variable to look at - even if not included in the final results
    

head(ckd_data)







,albumin_level,classification,appetite
,<dbl>,<chr>,<fct>
1,1,ckd,0
2,0,ckd,0
3,4,ckd,1
4,3,ckd,0
5,0,ckd,1
6,0,notckd,0


In [31]:
#for the sake of interest - look at the sum of albumin level in respect to appetite
#1 for presence of appetite, 0 for no appetite
appetite <- ckd_data %>%
    select(appetite, albumin_level)%>%
    group_by(appetite)%>%
    summarize(sum = sum(albumin_level))

head(appetite)


appetite,sum
<fct>,<dbl>
0,125
1,71


In [ ]:
#plotting the sample distribution for albumin
albumin_plot <- ckd_data %>%
    ggplot(aes(x = albumin_level))+
    geom_histogram(bins = 5)+
    xlab("Albumin Level")+
    ggtitle("Sample Distribution of Albumin Levels")

albumin_plot

In [24]:
#computing point estimates(mean)
xbar <- ckd_data %>%
    select(classification, albumin_level)%>%
    group_by(classification)%>%
    summarize(mean = mean(albumin_level))

xbar

classification,mean
<chr>,<dbl>
ckd,1.53125
notckd,0.00000


In [28]:


#computing observed test statistic (difference in mean)
diff <- xbar%>%
    mutate(diff = mean[1] - mean[2])%>%
    select(diff)

diff


diff
<dbl>
1.53125
1.53125
